In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

In [3]:
sess = tf.InteractiveSession()

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [6]:
#weights and biases
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [7]:
sess.run(tf.global_variables_initializer())

In [8]:
#predicted value
y = tf.matmul(x,W)+b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits = y))

In [9]:
#train model
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})

In [10]:
#Evaluation
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9185


##  Build a Multilayer convolutional network

### ReLU neurons
$$ f(x) = max(0,x)$$
$$ f(x) = ln(1+e^x)$$

### Weight Initialization

In [11]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

In [12]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

### Convolution and Pooling 

##### vanilla version: stride of 1 and 0 padded (output is the same size as input)

In [23]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding='SAME')

In [24]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [25]:
#lets see it in tensorboard
writer = tf.summary.FileWriter('/home/abhijit/Desktop/abhi/logs', graph=tf.get_default_graph())

### First convolution layer

##### Convolution followed by max pooling. The covoluion will compute 32 features for each 5x5 patch. Its weight ensor will have a shape of [5,5,1,32]. The 5x5 is the patch size, 1 is the number of input channels, and 32 is the number of output channel

In [26]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

##### For applying the layer, first reshape x to a 4d tensorm with the second and third dimensions corresponding to image width and height, and final dimension corresponding to number of color channers 

In [27]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

##### convolve x_image with the weight tensor, add the bias, apply the ReLU and finally max pool. The max_pool_2x2 will reduce the image size to 14x14 

In [28]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolution Layer

In [29]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Densely Connected Layer

##### Now as the image size has been reduced to 7x7, add a fully-connected layer with 1024 neurons to allow processing of entire image. Reshape the tensor from pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply ReLU 

In [30]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

### Dropout 

##### Placeholder for the probability that a neuron's output is kept duing dropout, This allows to turn dropout on during training, and turn it off during testing. 

In [31]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout layer

In [32]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

In [33]:
y_conv = tf.matmul(h_fc1_drop, W_fc2)+b_fc2

### Training and Evaluation 

In [34]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())


for i in range(20000):
    batch = mnist.train.next_batch(50)
    if(i%100 == 0):
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training acuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0}))

step 0, training acuracy 0.14
step 100, training acuracy 0.82
step 200, training acuracy 0.84
step 300, training acuracy 0.86
step 400, training acuracy 0.9
step 500, training acuracy 1
step 600, training acuracy 0.92
step 700, training acuracy 0.98
step 800, training acuracy 0.94
step 900, training acuracy 0.98
step 1000, training acuracy 0.96
step 1100, training acuracy 1
step 1200, training acuracy 1
step 1300, training acuracy 0.94
step 1400, training acuracy 1
step 1500, training acuracy 1
step 1600, training acuracy 0.94
step 1700, training acuracy 0.98
step 1800, training acuracy 1
step 1900, training acuracy 1
step 2000, training acuracy 0.98
step 2100, training acuracy 1
step 2200, training acuracy 1
step 2300, training acuracy 0.98
step 2400, training acuracy 0.96
step 2500, training acuracy 1
step 2600, training acuracy 0.96
step 2700, training acuracy 1
step 2800, training acuracy 0.98
step 2900, training acuracy 0.98
step 3000, training acuracy 1
step 3100, training acurac